In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
cotor = pd.read_csv('../data/processed/co_author_2.csv')

In [8]:
article = pd.read_csv('../data/econometry/base_1_todos.csv')

In [26]:
cite = pd.read_csv('../data/processed/cites.csv')

In [12]:
first = article.groupby('Short-Id').year.first().reset_index()

In [15]:
first = first.dropna()

In [21]:
graphs = {}
for y in sorted(cotor.year.unique()):
    chunk = cotor[cotor.year==y].groupby(['author1', 'author2']).size().rename('weight').reset_index()
    graphs[y] = nx.from_pandas_edgelist(chunk, source='author1', target='author2', edge_attr='weight')

In [31]:
super_cited = {}
for y in sorted(cotor.year.unique()):
    chunk = cite[cite.s_year==y]
    chunk = chunk.groupby('source').size().rename('cites').reset_index()
    sc = chunk[chunk.cites >= chunk.cites.mean() + 1.5 * (
        chunk.cites.quantile(.75) - chunk.cites.quantile(.25))].source.unique()
    super_cited[y] = sc

In [38]:
if set([]):
    print(8)

In [82]:
first = first[first.year.isin(cotor.year.unique())]

In [78]:
data = []
for row in first.iterrows():
    author = row[1]['Short-Id']
    y = int(row[1]['year'])
    colabs = []
    inters = []
    flag = np.nan
    for year in range(y, min(y+3, 2019)):
        if year in graphs:
            if author in graphs[year]:
                if flag != 1:
                    flag = 0
                colabs.append(graphs[year].degree(author))
                inters.append(graphs[year].degree(author, weight='weight'))
                if set(super_cited[year]) & set(list(graphs[year].neighbors(author))):
                    flag = 1
    data.append([author, y, np.mean(colabs), np.mean(inters), flag])

In [79]:
three = pd.DataFrame(data, columns=['Short-Id', 'year', 'colaborators', 'interactions', 'super_cited_colab'])

In [86]:
three = three.dropna()

In [88]:
three.to_csv('../data/econometry/three_variables.csv', index=False)

In [89]:
three.head()

,Short-Id,year,colaborators,interactions,super_cited_colab
2,paa11,2003,2.0,2.0,0.0
4,paa13,2004,3.0,4.0,1.0
5,paa2,2004,1.0,1.5,0.0
7,paa22,2012,1.0,1.0,0.0
9,paa25,2015,1.0,1.0,0.0


In [87]:
three.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30989 entries, 2 to 50666
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Short-Id           30989 non-null  object 
 1   year               30989 non-null  int64  
 2   colaborators       30989 non-null  float64
 3   interactions       30989 non-null  float64
 4   super_cited_colab  30989 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.4+ MB
